In [1]:
!javac de/unima/ki/anyburl/*.java -d build

In [2]:
!jar cfv AnyBURL-23-1.jar -C build .

added manifest
adding: de/(in = 0) (out= 0)(stored 0%)
adding: de/unima/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/Apply.class(in = 9153) (out= 5041)(deflated 44%)
adding: de/unima/ki/anyburl/Eval.class(in = 4680) (out= 2438)(deflated 47%)
adding: de/unima/ki/anyburl/exceptions/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/exceptions/RuleFunctionalityBasicSupportOnly.class(in = 567) (out= 384)(deflated 32%)
adding: de/unima/ki/anyburl/threads/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/threads/RuleWriterAsThread.class(in = 4034) (out= 2036)(deflated 49%)
adding: de/unima/ki/anyburl/threads/Predictor.class(in = 1523) (out= 657)(deflated 56%)
adding: de/unima/ki/anyburl/threads/Scorer.class(in = 4873) (out= 2552)(deflated 47%)
adding: de/unima/ki/anyburl/eval/(in = 0) (out= 0)(stored 0%)
adding: de/unima/ki/anyburl/eval/Candidate.class(in = 935) 

In [3]:
import pandas as pd
import zipfile
import os
from tqdm import tqdm
import time

import platform

import sys

import cpuinfo

import psutil


In [4]:
os.cpu_count()

32

In [5]:
psutil.virtual_memory()

svmem(total=99980713984, available=95995777024, percent=4.0, used=3511087104, free=96205324288, active=780619776, inactive=115486720, buffers=11022336, cached=253280256, shared=2793472, slab=283111424)

In [6]:
model_name = 'anyburl_io'

In [7]:
directory = model_name+"_entire_graph"

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory anyburl_io_entire_graph created successfully!


In [8]:
train_prop = f'PATH_TRAINING = MSCallGraph_train_transductive_io.tsv\nPATH_OUTPUT   = rules\nSNAPSHOTS_AT = 10,50,100\nWORKER_THREADS = 30\nSAFE_PREFIX_MODE = true'
f = open("train_prop.txt", "w")
f.write(train_prop)
f.close()
!java -Xmx1G -cp AnyBURL-23-1.jar de.unima.ki.anyburl.Learn train_prop.txt

reading params from file train_prop.txt
* read 41970 triples
* indexed 10000 triples
* indexed 20000 triples
* indexed 40000 triples
* set up index for 8 relations, 16276 head entities, and 3562 tail entities
* set up list structure for randomized access searches uring rule learning ...  done
* precomputing random starting points for each relation/direction for the beam search ... done.
* creating worker thread #0
* creating worker thread #1
* creating worker thread #2
* creating worker thread #3
* creating worker thread #4
* creating worker thread #5
* creating worker thread #6
* creating worker thread #7
* creating worker thread #8
* creating worker thread #9
* creating worker thread #10
* creating worker thread #11
* creating worker thread #12
* creating worker thread #13
* creating worker thread #14
* creating worker thread #15
* creating worker thread #16
* creating worker thread #17
* creating worker thread #18
* creating worker thread #19
* creating worker thread #20
* creating 

In [9]:
start = time.time()

In [10]:
predict_prop = f'PATH_TRAINING = MSCallGraph_train_transductive_io.tsv\nPATH_VALID    = MSCallGraph_validation_transductive_io.tsv\nPATH_TEST     = MSCallGraph_test_transductive_io.tsv\nPATH_RULES    = rules-100\nPATH_OUTPUT   = preds-100\nWORKER_THREADS = 30\nTOP_K_OUTPUT = 100\nSAFE_PREFIX_MODE = true'
f = open("predict_prop.txt", "w")
f.write(predict_prop)
f.close()
!java -Xmx1G -cp AnyBURL-23-1.jar de.unima.ki.anyburl.Apply predict_prop.txt

* reading params from file predict_prop.txt
* writing prediction to preds-100
* read 41516 triples
* indexed 10000 triples
* indexed 20000 triples
* indexed 40000 triples
* set up index for 8 relations, 16275 head entities, and 3561 tail entities
* read 13078 triples
* indexed 10000 triples
* set up index for 8 relations, 4901 head entities, and 2151 tail entities
* read 10276 triples
* indexed 10000 triples
* set up index for 8 relations, 4302 head entities, and 2030 tail entities
* reading rules from rules-100, read 501865 rules
* applied confidence threshold of 1.0E-4 and reduced from 501865 to 501865 rules
* applying rules
* indexed 100000 rules for prediction
* indexed 200000 rules for prediction
* indexed 300000 rules for prediction
* indexed 400000 rules for prediction
* indexed 500000 rules for prediction
* indexed and sorted 501865 rules for using them to make predictions
* set up index structure covering rules for prediction for 8 relations
* creating worker threads #0 #1 #2 

In [11]:
eval_prop = f'PATH_TRAINING = MSCallGraph_train_transductive_io.tsv\nPATH_VALID    = MSCallGraph_validation_transductive_io.tsv\nPATH_TEST     = MSCallGraph_test_transductive_io.tsv\nPATH_PREDICTIONS = preds-100\nTOP_K = 100\nSAFE_PREFIX_MODE = true'
f = open("eval_prop.txt", "w")
f.write(eval_prop)
f.close()
evaluation_result = !java -Xmx1G -cp AnyBURL-23-1.jar de.unima.ki.anyburl.Eval eval_prop.txt
evaluation_result

['reading params from file eval_prop.txt',
 '* read 41516 triples',
 '* indexed 10000 triples',
 '* indexed 20000 triples',
 '* indexed 40000 triples',
 '* set up index for 8 relations, 16275 head entities, and 3561 tail entities',
 '* read 10276 triples',
 '* indexed 10000 triples',
 '* set up index for 8 relations, 4302 head entities, and 2030 tail entities',
 '* read 13078 triples',
 '* indexed 10000 triples',
 '* set up index for 8 relations, 4901 head entities, and 2151 tail entities',
 '* loading result set at preds-100',
 '0.4546   0.5808   0.6968   0.5384']

In [12]:
testing_evaluation_duration = time.time() - start

In [13]:
evaluation_result[-1].split()

['0.4546', '0.5808', '0.6968', '0.5384']

In [14]:

        


metric_names = ['hits_at_1','hits_at_3','hits_at_10','mrr']

df = pd.DataFrame([evaluation_result[-1].split()],columns=metric_names)
df.to_csv(f"{model_name}_entire_graph/{model_name}.csv")

    
display(df)


,hits_at_1,hits_at_3,hits_at_10,mrr
0,0.4546,0.5808,0.6968,0.5384


In [15]:
infodict = {}
infodict['parameters bytes'] = 1e+9
infodict['number parameters'] = 377728 #number of rules
infodict['training duration'] = 100
# infodict['training evaluation duration'] = training_evaluation_duration
# infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [16]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}_entire_graph/{model_name}_information.csv")
print(info_df)

                           name  \
0              parameters bytes   
1             number parameters   
2             training duration   
3   testing evaluation duration   
4         Operating system name   
5      Operating system version   
6        Processor architecture   
7                Python version   
8          Processor model name   
9              Number cpu cores   
10        Total physical memory   

                                                value  
0                                        1000000000.0  
1                                              377728  
2                                                 100  
3                                           59.603945  
4                                               Linux  
5                         3.10.0-1160.88.1.el7.x86_64  
6                                              x86_64  
7   3.10.9 (main, Mar  8 2023, 10:47:38) [GCC 11.2.0]  
8            Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz  
9                  

In [17]:
def zip_folder(folder_path, output_path):
     with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name+"_entire_graph"
output_path = f'{folder_path}.zip'

zip_folder(folder_path, output_path)